# Fine-Tuning StableDiffusion XL with DreamBooth

Over the past few years Generative AI models have popped up everywhere - from creating realistic responses to complex questions, to generating images and music to impress art critics around the globe. In this notebook we use the Hugging Face [Stable Diffusion XL (SDXL)](https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0) model to create images from text prompts. You'll see how to import the SDXL model and use it to generate an image. 

From there, you'll see how you can fine-tune the model using [DreamBooth](https://huggingface.co/docs/diffusers/training/dreambooth), a method for easily fine-tuning a text-to-image model. We'll use a small number of photos of [Toy Jensen](https://blogs.nvidia.com/blog/2022/12/22/toy-jensen-jingle-bells/) in this notebook to fine-tune SDXL. This will allow us to generate new images that include Toy Jensen! 

After that, you'll have the chance to fine-tune the model on your own images. Perhaps you want to create an image of you at the bottom of the ocean, or in outer space? By the end of this notebook you will be able to! 

**IMPORTANT:** This project will utilize additional third-party open source software. Review the license terms of these open source projects before use. Third party components used as part of this project are subject to their separate legal notices or terms that accompany the components. You are responsible for confirming compliance with third-party component license terms and requirements.

### Stable Diffusion XL Model

First, we import the classes and libraries we need to run the notebook.

In [ ]:
# !echo "BEFORE:: Virtual ENV - $VIRTUAL_ENV"
# !echo "         PATH: $PATH"
# !python -m venv nvidia-usecase
# !source nvidia-usecase/bin/activate
# !echo "AFTER:: Virtual ENV - $VIRTUAL_ENV"
# !echo "        PATH: $PATH"

# !python -m venv nvidia-usecase
# !source nvidia-usecase/bin/activate

!pip uninstall -y -q optimum["onnxruntime"] optimum[exporters] datasets evaluate
!pip uninstall -y -q diffusers datasets evaluate huggingface-hub torch-model-archiver

!pip install --upgrade pip
!pip install -q -r ../requirements.txt

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, DiffusionPipeline

Next, from the Hugging Face `diffusers` library, we create a `StableDiffusionXLPipeline` object from the SDXL base model. 

In [ ]:
model_id="stabilityai/stable-diffusion-xl-base-1.0"

!echo ""
!echo "Using [{model_id}] as the pre-trained model for this demo"
!echo ""

pipe = StableDiffusionXLPipeline.from_pretrained(model_id, torch_dtype=torch.float16, variant="fp16", use_safetensors=True)
pipe.to("cuda")
# pipe.enable_model_cpu_offload()

Let's use the SDXL model to generate an image. 

In [ ]:
prompt = "toy jensen in space"
image = pipe(prompt=prompt).images[0]

image

Hmmm, looks like the Hugging Face SDXL model doesn't know about Toy Jensen! Imagine that! 

✅ Try using the SDXL model to generate some other images by editing the text in the first line of the cell above. 


## Fine-Tuning the model with DreamBooth

Fine-Tuning is used to train an existing Machine Learning Model, given new information. In our case, we want to teach the SDXL model about Toy Jensen. This will allow us to create the perfect image of Toy Jensen in Space!

[DreamBooth](https://arxiv.org/abs/2208.12242) provides a way to fine-tune a text-to-image model using only a few images. Let's use this to tune our SDXL Model so that it knows about Toy Jensen!

We have 8 photos of Toy Jensen in our dataset - let's take a look at one of them.

In [ ]:
from IPython.display import Image

display(Image(filename='../data/toy-jensen/tj1.png'))

In [ ]:
#Lets clone `diffusers` repo and use the correct versions of huggingface cli and torch
!rm -rf repos/diffusers
!mkdir -p repos

!git clone https://github.com/huggingface/diffusers repos/diffusers
!cd repos/diffusers && git checkout v0.21.4
!pip install -q peft==0.9.0 huggingface_hub[cli,torch]==0.21.4

Now we can use Hugging Face and DreamBooth to fine-tune this model. To do this we create a config, then specify some flags like an instance prompt, a resolution and a number of training steps for the fine-tuning algorithm to run. 

In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()

In [ ]:
import os
import torch

# Set PYTORCH_CUDA_ALLOC_CONF
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1024"

# Print total memory and other device properties
print(torch.cuda.get_device_properties(0).total_memory)
print(torch.cuda.get_device_properties(0))

In [ ]:
output_dir = "../models/tuned-toy-jensen"
output_lora_dir = output_dir + "/lora"   # pytorch_lora_weights.safetensors will be generated in this dir

!echo ""
!echo "Using [{model_id}] as the pre-trained model for this demo"
!echo "  output dir: [{output_dir}]"
!echo ""

torch.cuda.empty_cache()

!accelerate launch ./repos/diffusers/examples/dreambooth/train_dreambooth_lora_sdxl.py \
  --pretrained_model_name_or_path={model_id}  \
  --instance_data_dir=../data/toy-jensen \
  --output_dir={output_lora_dir} \
  --mixed_precision="bf16" \
  --instance_prompt="a photo of toy jensen" \
  --resolution=768 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=100 \
  --seed="0" \
  --resume_from_checkpoint=latest

Now that the model is fine-tuned, let's tell our notebook where to find it.

In [ ]:
# Uncomment next 3 statements if re-running steps from this cell (in case of OOM and Kernel disconnect)
model_id="stabilityai/stable-diffusion-xl-base-1.0"
output_dir = "../models/tuned-toy-jensen"
output_lora_dir = output_dir + "/lora"   # pytorch_lora_weights.safetensors will be generated in this dir

pipe = StableDiffusionXLPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")
pipe.load_lora_weights(output_lora_dir)

Finally, we can use our fine-tuned model to create an image with Toy Jensen in it. Let's give it a go! 

In [ ]:
image = pipe("A picture of toy jensen in space", num_inference_steps=75).images[0]

image

Wow - look at him go! 

In [ ]:
output_model_dir = output_dir + "/model"
print(f"Saving fine-tuned model to: [{output_model_dir}]")

pipe.save_pretrained(output_model_dir)

In [ ]:
# Uncomment next 3 statements if re-running steps from this cell (in case of OOM and Kernel disconnect)
# output_dir = "../models/tuned-toy-jensen"
# output_model_dir = output_dir + "/model"
# onnx_output_model_dir = output_dir + "/sd_xl_onnx"
# mar_output_model_dir = output_dir + "/mar/archive"

!echo
!echo "Data in model output dir [{output_model_dir}]"
!ls -lh {output_model_dir}

!echo "Fine-tuned model safetensor files in [{output_model_dir}]"
!find ../models/tuned-toy-jensen/model -name "*safetensors" -exec ls -lh {} \;

!echo "Fine-tuned model onnx files in [{onnx_output_model_dir}]"
!find ../models/tuned-toy-jensen/sd_xl_onnx -name "*onnx*" -type f -exec ls -lh {} \;

# !echo "Model ARchive file in [{mar_output_model_dir}]"
# !find ../models/tuned-toy-jensen/mar/archive -type f -exec ls -lh {} \;

# To see the whole model directory uncomment next statement
# !find ../models/tuned-toy-jensen/model

In [ ]:
# -------------------------------------------------
# Import the transfer notebook to upload our pytorch model to minio
# -------------------------------------------------
%run ./Xfer-to-minio.ipynb

# Uncomment next statement if running only this cell (in case of OOM and Kernel disconnect)
output_model_dir = "../models/tuned-toy-jensen/model"
prefix = "torchserve-no_zip/fine-tuned"
s3_env: S3Env = init_env()
minio_client: Minio = init_minio(s3_env)

dir_model = MinioBucketMeta(model_data_dir=output_model_dir,
                            bucket_name=s3_env.bucket_name,
                            client=minio_client,
                            prefix=prefix)
upload_files(dir_model)

In [ ]:
# -------------------------------------------------
# Generate the "mar" file for pytorch/torchserve
# -------------------------------------------------

# !./create-mar.sh "/opt/app-root/src/demo/sgahlot-nvidia-usecase"
!./create-mar-without_zip.sh "/opt/app-root/src/demo/sgahlot-nvidia-usecase"

In [ ]:
# -------------------------------------------------
# Import the transfer notebook to upload our pytorch model to minio
# -------------------------------------------------
%run ./Xfer-to-minio.ipynb

# Uncomment next statement if running only this cell (in case of OOM and Kernel disconnect)
output_dir = "../models/tuned-toy-jensen"
# mar_output_model_dir = output_dir + "/gen-mar/archive"
mar_output_model_dir = output_dir + "/gen-mar/archive-without-zip"
prefix = "torchserve-no_zip"

s3_env: S3Env = init_env()
minio_client: Minio = init_minio(s3_env)

dir_model = MinioBucketMeta(model_data_dir=mar_output_model_dir,
                            bucket_name=s3_env.bucket_name,
                            client=minio_client,
                            prefix=prefix,
                            exclude_dirs_set=['logs', 'workloads'])
upload_files(dir_model)

In [ ]:
# -------------------------------------------------
# Convert the model to a single ONNX file - NOT WORKING in OpenVino YET
# -------------------------------------------------
!pip install -q optimum["onnxruntime"]
!pip install -q optimum[exporters]
!pip install --upgrade diffusers

onnx_output_model_dir = output_dir + "/sd_xl_onnx"

# If using a single safetensors file (created using the following command in output_dir)
# python ../../code/repos/diffusers-main/scripts/convert_diffusers_to_original_sdxl.py \
#      --model_path ./model/ \
#      --checkpoint_path ./model-single/single.safetensors
# output_model_dir = output_dir + "/model-single"

!optimum-cli export onnx --model {output_model_dir} \
       --task text-to-image --device cuda --dtype fp16 \
       --framework pt --cache_dir ./sd_xl_onnx_cache \
       --monolith {onnx_output_model_dir}/
       # Specify the following option if using a single onnx file
       # --library diffusers

In [ ]:
# -------------------------------------------------
# Import the transfer notebook to upload onnx model to minio
# -------------------------------------------------
%run ./Xfer-to-minio.ipynb

prefix = "fine-tuned-onnx"
s3_env: S3Env = init_env()
minio_client: Minio = init_minio(s3_env)

dir_model = MinioBucketMeta(model_data_dir=onnx_output_model_dir,
                            bucket_name=s3_env.bucket_name,
                            client=minio_client,
                            prefix=prefix)
# print(f'\n -> ONNX model data dir: {dir_model.model_data_dir}, bucket={dir_model.bucket_name}')
upload_files(dir_model)

In [ ]:
# -------------------------------------------------
# To read the model from save model directory
# -------------------------------------------------

# # output_dir = "../models/tuned-toy-jensen"
# # output_model_dir = output_dir + "/model"

# pipe_1 = StableDiffusionXLPipeline.from_pretrained(output_model_dir, torch_dtype=torch.float16, use_safetensors=True, subfolder="scheduler")
# pipe_1 = pipe.to("cuda")
# pipe_1.load_lora_weights(output_lora_dir)

In [ ]:
# -------------------------------------------------
# To read the model from a single safetensors file
# -------------------------------------------------

# !pip install omegaconf

# output_dir = "../models/tuned-toy-jensen"

# pipe = StableDiffusionXLPipeline.from_single_file(output_dir + "/single.safetensors", torch_dtype=torch.float16)
# pipe = pipe.to("cuda")